# **Avance 2. Series de tiempo univariadas:**
## **Universidad Nacional de Colombia.**
## **Docente:**

  - [Sergio Alejandro Calderón Villanueva](sacalderonv@unal.edu.co)

## **Autores:**

  - [Michel Mendivenson Barragán Zabala](mbarraganz@unal.edu.co)
  - [Sharon Juliana Zambrano Ortiz](shzambranoo@unal.edu.co)
  - [Nicolás Alejandro Quenguan Parra](naquenguanp@unal.edu.co)
  - [Aryin Jair Torres Quiñone](artorresq@unal.edu.co)

---
**Nota 1:** Los avances estaban siendo realizados en archivos de Rmarkdown `.Rmd`. Sin embargo, como la intención de este avance es incluir algunos modelos de aprendizaje profundo (Redes neuronales multicapa y redes recurrentes) con el fin de no utilizar recursos de computo de los equipos de cada uno de los autores se entregará el avance 2 como un archivo de jupyter notebook `.ipynb` que además de permitir el uso de computación en la nube para tareas pesadas usando Google COllab cuando los recursos de forma local sean reducidos también (Por ser `python`) permite construir chunks que usen código de `R` y `python` cuyos objetos tienen la posibilidad de interactuar entre sí (Veáse: [R and Python in one Jupyter notebook](https://stackoverflow.com/questions/39008069/r-and-python-in-one-jupyter-notebook))

**Nota 2:** Si se está editando este documento desde Google Collab, no es posible hacer `commit` directamente al repositorio. En cambio, hay que dar en la opción `File > Save a copy in GitHub`.

El siguiente código habilita el uso de código de `R` y `python` en simultáneo:

In [2]:
%load_ext rpy2.ipython

Ahora se cargan los datos usando `R`:

In [3]:
%%R
leche = read.csv("https://raw.githubusercontent.com/Mendivenson/Series-de-tiempo/main/Datos/Volumen%20de%20Acopio%20Total%20%20(2008%20-%202023).csv")
leche = leche[, c('Periodo', 'NACIONAL')]

Y exporta el objeto a `python`:

In [ ]:
%%R -o leche

Cualquier chunk de código que inicie con `%%R` será código `R` por lo que el siguiente código es 100% `python`:

In [7]:
leche

,Periodo,NACIONAL
1,Jan-08,2.458355e+08
2,Feb-08,2.130557e+08
3,Mar-08,2.175541e+08
4,Apr-08,2.224843e+08
5,May-08,2.386657e+08
...,...,...
188,Aug-23,2.979559e+08
189,Sep-23,2.845475e+08
190,Oct-23,2.876751e+08
191,Nov-23,2.809905e+08


from matplotlib import pyplot as plt
leche['NACIONAL'].plot(kind='hist', bins=20, title='NACIONAL')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
leche['NACIONAL'].plot(kind='line', figsize=(8, 4), title='NACIONAL')
plt.gca().spines[['top', 'right']].set_visible(False)

In [16]:
import pandas as pd
print(prueba.head(10))

   Periodo    ANTIOQUIA  BOGOTA.DC       BOYACÁ      CALDAS       CAUCA  \
1   Jan-08  73609498.60        NaN  18843690.39  4785911.00   892856.00   
2   Feb-08  63957645.89        NaN  17406274.19  4737965.12   962645.00   
3   Mar-08  66905147.00        NaN  16962408.50  5141336.00  1069200.79   
4   Apr-08  67383870.97        NaN  17853955.43  5034305.37  1046277.00   
5   May-08  79123350.00        NaN  16181284.00  5213326.00  1122881.00   
6   Jun-08  66736756.95        NaN  18538776.84  4945978.84  1095608.00   
7   Jul-08  67672960.88        NaN  19325822.07  4738868.42  1127330.00   
8   Aug-08  77574090.86        NaN  19341161.00  4828351.00  1233870.00   
9   Sep-08  62399349.00        NaN  20722673.00  4325502.00  1171302.00   
10  Oct-08  72104332.43        NaN  17816142.57  5331264.00  1134684.00   

    CUNDINAMARCA      NARIÑO    QUINDÍO   RISARALDA  ...  LA.GUAJIRA  \
1    70912463.96  9625806.00  3311389.4   940720.00  ...   128624.00   
2    56688151.32  9052061.00  

In [13]:
%%R
head(prueba[,c('Periodo', 'NACIONAL')])

  Periodo  NACIONAL
1  Jan-08 245835529
2  Feb-08 213055728
3  Mar-08 217554072
4  Apr-08 222484264
5  May-08 238665668
6  Jun-08 241011381
